In [1]:
import pandas as pd
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.special import logit, expit
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

In [2]:
algorithm = 'RF'

In [3]:
torch.set_default_dtype(torch.float64)

In [4]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                #lengthscale_prior = gpytorch.priors.NormalPrior(10,100),
                lengthscale_constraint = gpytorch.constraints.Interval(1e1,1e4),
                alpha_constraint = gpytorch.constraints.Interval(1e-2,1e2),
                #ard_num_dims = 2,
                #outputscale_prior = gpytorch.priors.NormalPrior(10,100),
                #outputscale_constraint = gpytorch.constraints.Interval(1e2,1e5),
            ))
        # length_scale_bounds = [1e-10,1e10],alpha_bounds = [1e-10,1e10]
        #self.covar_module.initialize_from_data(train_x, train_y)
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)            

In [5]:
global num_classifications
num_classifications = 50
X = np.linspace(0,num_classifications,num_classifications+1)

In [6]:
def analytical_fit(x,a,b):
    return ((1+b*x)*np.exp(a*x)-1)/((1+b)*np.exp(a)-1)

In [7]:
precision = 6
noise_level_NS = 0.0001/num_classifications
noise_level_REM = 0.0001/num_classifications

In [8]:
Bayes = pd.read_csv('BayesFactors.csv',delimiter=',')
Bayes_factors = Bayes['BayesFactor'] / np.sum(Bayes['BayesFactor'])

P_NS_given_X1pX2_num = pd.DataFrame({'X1+X2': X/num_classifications})#.astype({'X1+X2': np.int32})  
P_REM_given_X2_num = pd.DataFrame({'X2': X/num_classifications})#.astype({'X2': np.int32})
                                                        
P_NS_given_X1pX2_table = pd.DataFrame({'X1+X2': X/num_classifications})#.astype({'X1+X2': np.int32})
P_REM_given_X2_table =pd.DataFrame({'X2': X/num_classifications})#.astype({'X2': np.int32})

P_NS_given_X1pX2_table_an = pd.DataFrame({'X1+X2': X/num_classifications})#.astype({'X1+X2': np.int32})
P_REM_given_X2_table_an = pd.DataFrame({'X2': X/num_classifications})#.astype({'X2': np.int32})

In [9]:
for EOS_tag in Bayes['EOS']:
    test_file = 'matrix_events_'+algorithm+'_EOS-' + EOS_tag + '.csv'

    classification_results = pd.read_csv(test_file,delimiter=',')#.dropna(axis=1) #, nrows=20)

    probability_dataframe = pd.DataFrame()
    file_tag = test_file.split('.')[0]
    log_file =  'log-' + file_tag + '.txt'

    P_NS_tag = 'P_NS('+EOS_tag+')'

    probability_dataframe['LABEL'] = classification_results['HasNS_true'] + \
                                          classification_results['HasREM_true']
    probability_dataframe['X0'] = np.round(classification_results['f_0'] * num_classifications,0)
    probability_dataframe['X1'] = np.round(classification_results['f_1'] * num_classifications,0)
    probability_dataframe['X2'] = np.round(classification_results['f_2'] * num_classifications,0)
    probability_dataframe['X1+X2'] = np.round((classification_results['f_1'] + \
                                           classification_results['f_2']) * num_classifications,0)
    probability_dataframe = probability_dataframe.fillna(0)

    N_events = len(probability_dataframe)

    NS = probability_dataframe[probability_dataframe['LABEL']>0]
    N_NS = len(NS)
    P_NS = N_NS / N_events
    N_X1pX2 = np.array([len(probability_dataframe[probability_dataframe['X1+X2']==el]) for el in X])
    N_X1pX2_given_NS = np.array([len(NS[NS['X1+X2']==el]) for el in X])
    P_X1pX2 = N_X1pX2 / N_events
    P_X1pX2_given_NS = N_X1pX2_given_NS / N_NS
    P_NS_given_X1pX2 = np.divide(P_X1pX2_given_NS,P_X1pX2) * P_NS

    P_NS_given_X1pX2_num[P_NS_tag] = P_NS_given_X1pX2
    P_NS_given_X1pX2_num = P_NS_given_X1pX2_num.fillna(method='ffill')
    P_NS_given_X1pX2_num = P_NS_given_X1pX2_num.round({P_NS_tag:precision})
        
    P_NS_given_X1pX2_fit = P_NS_given_X1pX2_num 
    
    P_NS_given_X1pX2_fit.loc[P_NS_given_X1pX2_fit[P_NS_tag] == 0.0,P_NS_tag] =  \
        P_NS_given_X1pX2_fit[P_NS_tag].apply(lambda x: noise_level_NS*np.random.rand())
    P_NS_given_X1pX2_fit.loc[P_NS_given_X1pX2_fit[P_NS_tag] == 1.0,P_NS_tag] = \
        P_NS_given_X1pX2_fit[P_NS_tag].apply(lambda x: 1.0-noise_level_NS*np.random.rand())
    
    P_NS_given_X1pX2_logit = logit(P_NS_given_X1pX2_fit[P_NS_tag].values)
       
    P_NS_given_X1pX2_logit = savgol_filter(P_NS_given_X1pX2_logit,2,1) 

    P_NS_given_X1pX2_logit_array = P_NS_given_X1pX2_logit.reshape(-1,1)
    P_NS_given_X1pX2_scaler = preprocessing.StandardScaler().fit(P_NS_given_X1pX2_logit_array)
    P_NS_given_X1pX2_scaled = P_NS_given_X1pX2_scaler.transform(P_NS_given_X1pX2_logit_array).flatten()

    P_NS_given_X1pX2_train_x = torch.from_numpy(np.linspace(0,num_classifications,len(P_NS_given_X1pX2_scaled))).to(torch.float64)
    P_NS_given_X1pX2_train_y = torch.from_numpy(P_NS_given_X1pX2_scaled).to(torch.float64)
    P_NS_given_X1pX2_test_x = torch.from_numpy(X).to(torch.float64)

    likelihood_P_NS_given_X1pX2 = gpytorch.likelihoods.GaussianLikelihood(num_tasks=2)
    model_P_NS_given_X1pX2 = MultitaskGPModel(P_NS_given_X1pX2_train_x, P_NS_given_X1pX2_train_y, \
                                      likelihood_P_NS_given_X1pX2)
    model_P_NS_given_X1pX2.train()
    likelihood_P_NS_given_X1pX2.train()

    optimizer_P_NS_given_X1pX2 = torch.optim.Adam([
    {'params': model_P_NS_given_X1pX2.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=1)

    mll_P_NS_given_X1pX2 = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood_P_NS_given_X1pX2, \
                                                            model_P_NS_given_X1pX2)

    n_iter_P_NS_given_X1pX2 = 100
    for i in range(n_iter_P_NS_given_X1pX2):
        optimizer_P_NS_given_X1pX2.zero_grad()
        output_P_NS_given_X1pX2 = model_P_NS_given_X1pX2(P_NS_given_X1pX2_train_x)
        loss_P_NS_given_X1pX2 = -mll_P_NS_given_X1pX2(output_P_NS_given_X1pX2, P_NS_given_X1pX2_train_y).sum()
        loss_P_NS_given_X1pX2.backward()
        optimizer_P_NS_given_X1pX2.step()

    model_P_NS_given_X1pX2.eval()
    likelihood_P_NS_given_X1pX2.eval()  

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        P_NS_given_X1pX2_predictions = likelihood_P_NS_given_X1pX2(model_P_NS_given_X1pX2(P_NS_given_X1pX2_test_x))
        P_NS_given_X1pX2_mean = P_NS_given_X1pX2_predictions.mean
        lower_P_NS_given_X1pX2, upper_P_NS_given_X1pX2 = P_NS_given_X1pX2_predictions.confidence_region()

    P_NS_given_X1pX2_fitted = P_NS_given_X1pX2_scaler.inverse_transform(P_NS_given_X1pX2_mean.numpy().reshape(-1,1))
    P_NS_given_X1pX2_expit = expit(P_NS_given_X1pX2_fitted)
    P_NS_given_X1pX2_expit = P_NS_given_X1pX2_expit.reshape(num_classifications+1,-1)
    P_NS_given_X1pX2_table[P_NS_tag] = P_NS_given_X1pX2_expit   

    popt_NS,pcov_NS = curve_fit(analytical_fit,P_NS_given_X1pX2_num['X1+X2'],
                                savgol_filter(P_NS_given_X1pX2_num[P_NS_tag],2,1))
    analytical_fit_NS = analytical_fit(P_NS_given_X1pX2_num['X1+X2'],*popt_NS)

    P_NS_given_X1pX2_table_an[P_NS_tag] = analytical_fit_NS

    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)

    ax.scatter(P_NS_given_X1pX2_num['X1+X2'],P_NS_given_X1pX2_num[P_NS_tag],cmap='viridis')
    ax.scatter(P_NS_given_X1pX2_table['X1+X2'],P_NS_given_X1pX2_table[P_NS_tag],cmap='viridis')
    ax.scatter(P_NS_given_X1pX2_table_an['X1+X2'],analytical_fit_NS,cmap='viridis')

    plt.xlabel('X1+X2')
    plt.ylabel('P_NS')
    plt.minorticks_on()
    plt.grid(which='both')
    plt.savefig(algorithm+'_P_NS_given_X1pX2-'+EOS_tag+'-num.png',dpi=300,facecolor='w')
    plt.close()

    probability_dataframe_NS = probability_dataframe[probability_dataframe.LABEL != 0]
    P_REM_tag = 'P_REM('+EOS_tag+')'

    REM = probability_dataframe_NS[probability_dataframe_NS['LABEL']==2]

    N_events_NS = len(probability_dataframe_NS)
    N_REM = len(REM)
    P_REM = N_REM / N_events_NS
    N_X2 = np.array([len(probability_dataframe_NS[probability_dataframe_NS['X2']==el]) for el in X])
    N_X2_given_REM = np.array([len(REM[REM['X2']==el]) for el in X])
    P_X2 = N_X2 / N_events_NS
    P_X2_given_REM = N_X2_given_REM / N_REM
    P_REM_given_X2 = np.divide(P_X2_given_REM,P_X2) * P_REM

    P_REM_given_X2 = P_NS_given_X1pX2 * P_REM_given_X2

    P_REM_given_X2_num[P_REM_tag] =  P_REM_given_X2
    
    
    P_REM_given_X2_num = P_REM_given_X2_num.fillna(method='ffill')
    P_REM_given_X2_num = P_REM_given_X2_num.round({P_REM_tag:precision})

    P_REM_given_X2_fit = P_REM_given_X2_num 

    P_REM_given_X2_fit.loc[P_REM_given_X2_fit[P_REM_tag] == 0.0,P_REM_tag] =  \
        P_REM_given_X2_fit[P_REM_tag].apply(lambda x: noise_level_REM*np.random.rand())
    P_REM_given_X2_fit.loc[P_REM_given_X2_fit[P_REM_tag] == 1.0,P_REM_tag] = \
        P_REM_given_X2_fit[P_REM_tag].apply(lambda x: 1.0-noise_level_REM*np.random.rand())

    P_REM_given_X2_logit = logit(P_REM_given_X2_fit[P_REM_tag].values)

    P_REM_given_X2_logit = savgol_filter(P_REM_given_X2_logit,2,1) 
    
    P_REM_given_X2_logit_array = P_REM_given_X2_logit.reshape(-1,1)
    P_REM_given_X2_scaler = preprocessing.StandardScaler().fit(P_REM_given_X2_logit_array)
    P_REM_given_X2_scaled = P_REM_given_X2_scaler.transform(P_REM_given_X2_logit_array).flatten()

    P_REM_given_X2_train_x = torch.from_numpy(np.linspace(0,num_classifications,len(P_REM_given_X2_scaled))).to(torch.float64)
    P_REM_given_X2_train_y = torch.from_numpy(P_REM_given_X2_scaled).to(torch.float64)
    P_REM_given_X2_test_x = torch.from_numpy(X).to(torch.float64)

    likelihood_P_REM_given_X2 = gpytorch.likelihoods.GaussianLikelihood(num_tasks=2)
    model_P_REM_given_X2 = MultitaskGPModel(P_REM_given_X2_train_x, P_REM_given_X2_train_y, \
                                    likelihood_P_REM_given_X2)
    model_P_REM_given_X2.train()
    likelihood_P_REM_given_X2.train()

    optimizer_P_REM_given_X2 = torch.optim.Adam([
        {'params': model_P_REM_given_X2.parameters()},  # Includes GaussianLikelihood parameters
        ], lr=1)

    mll_P_REM_given_X2 = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood_P_REM_given_X2, \
                                                        model_P_REM_given_X2)

    n_iter_P_REM_given_X2 = 100
    for i in range(n_iter_P_REM_given_X2):
        optimizer_P_REM_given_X2.zero_grad()
        output_P_REM_given_X2 = model_P_REM_given_X2(P_REM_given_X2_train_x)
        loss_P_REM_given_X2 = -mll_P_REM_given_X2(output_P_REM_given_X2, P_REM_given_X2_train_y).sum()
        loss_P_REM_given_X2.backward()
        optimizer_P_REM_given_X2.step()

    model_P_REM_given_X2.eval()
    likelihood_P_REM_given_X2.eval()  

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        P_REM_given_X2_predictions = likelihood_P_REM_given_X2(model_P_REM_given_X2(P_REM_given_X2_test_x))
        P_REM_given_X2_mean = P_REM_given_X2_predictions.mean
        lower_P_REM_given_X2, upper_P_REM_given_X2 = P_REM_given_X2_predictions.confidence_region()

    P_REM_given_X2_fitted = P_REM_given_X2_scaler.inverse_transform(P_REM_given_X2_mean.numpy().reshape(-1,1))
    P_REM_given_X2_expit = expit(P_REM_given_X2_fitted)
    P_REM_given_X2_expit = P_REM_given_X2_expit.reshape(num_classifications+1,-1)
    P_REM_given_X2_table[P_REM_tag] = P_REM_given_X2_expit   
    
    popt_REM,pcov_REM = curve_fit(analytical_fit,P_REM_given_X2_num['X2'],
                                  savgol_filter(P_REM_given_X2_num[P_REM_tag],2,1))
    analytical_fit_REM = analytical_fit(P_REM_given_X2_num['X2'],*popt_REM)
    P_REM_given_X2_table_an[P_REM_tag] = analytical_fit_REM    
        
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)

    ax.scatter(P_REM_given_X2_num['X2'],P_REM_given_X2_num[P_REM_tag],cmap='viridis')
    ax.scatter(P_REM_given_X2_table['X2'],P_REM_given_X2_table[P_REM_tag],cmap='viridis')
    ax.scatter(P_REM_given_X2_table_an['X2'],analytical_fit_REM,cmap='viridis')

    plt.xlabel('X2')
    plt.ylabel('P_REM')
    plt.minorticks_on()
    plt.grid(which='both')
    plt.savefig(algorithm+'_P_REM_given_X2-'+EOS_tag+'-num.png',dpi=300,facecolor='w')
    plt.close()

    
P_NS_given_X1pX2_num.to_csv(algorithm+'_P_NS_given_X1pX2-num.csv',index=False,float_format='%.'+str(precision)+'f')
P_NS_given_X1pX2_table.to_csv(algorithm+'_P_NS_given_X1pX2-fit.csv',index=False,float_format='%.'+str(precision)+'f')
P_NS_given_X1pX2_table_an.to_csv(algorithm+'_P_NS_given_X1pX2-fit_an.csv',index=False,float_format='%.'+str(precision)+'f')    
    
P_REM_given_X2_num.to_csv(algorithm+'_P_REM_given_X2-num.csv',index=False,float_format='%.'+str(precision)+'f')
P_REM_given_X2_table.to_csv(algorithm+'_P_REM_given_X2-fit.csv',index=False,float_format='%.'+str(precision)+'f')
P_REM_given_X2_table_an.to_csv(algorithm+'_P_REM_given_X2-fit_an.csv',index=False,float_format='%.'+str(precision)+'f')


/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution
/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  wa

/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/home/cavaglia/.local/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:273: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/home/cava

In [10]:
P_NS_given_X1pX2_num_Bayes = P_NS_given_X1pX2_num.iloc[:,1:]
P_NS_given_X1pX2_num_Bayes.columns = range(len(Bayes_factors))
P_NS_given_X1pX2_num_Bayes = P_NS_given_X1pX2_num_Bayes.mul(Bayes_factors)
P_NS_given_X1pX2_num_Bayes.rename(columns='P_NS('+Bayes['EOS']+')',inplace=True)
P_NS_given_X1pX2_num_Bayes.insert(0,'X1+X2',P_NS_given_X1pX2_num['X1+X2'])

P_NS_given_X1pX2_num_Bayes.to_csv(algorithm+'_P_NS_given_X1pX2-BF-num.csv',index=False,float_format='%.'+str(precision)+'f')

P_NS_given_X1pX2_table_Bayes = P_NS_given_X1pX2_table.iloc[:,1:]
P_NS_given_X1pX2_table_Bayes.columns = range(len(Bayes_factors))
P_NS_given_X1pX2_table_Bayes = P_NS_given_X1pX2_table_Bayes.mul(Bayes_factors)
P_NS_given_X1pX2_table_Bayes.rename(columns='P_NS('+Bayes['EOS']+')',inplace=True)
P_NS_given_X1pX2_table_Bayes.insert(0,'X1+X2',P_NS_given_X1pX2_table['X1+X2'])

P_NS_given_X1pX2_table_Bayes.to_csv(algorithm+'_P_NS_given_X1pX2-BF-fit.csv',index=False,float_format='%.'+str(precision)+'f')   

P_NS_given_X1pX2_table_an_Bayes = P_NS_given_X1pX2_table_an.iloc[:,1:]
P_NS_given_X1pX2_table_an_Bayes.columns = range(len(Bayes_factors))
P_NS_given_X1pX2_table_an_Bayes = P_NS_given_X1pX2_table_an_Bayes.mul(Bayes_factors)
P_NS_given_X1pX2_table_an_Bayes.rename(columns='P_NS('+Bayes['EOS']+')',inplace=True)
P_NS_given_X1pX2_table_an_Bayes.insert(0,'X1+X2',P_NS_given_X1pX2_table_an['X1+X2'])

P_NS_given_X1pX2_table_an_Bayes.to_csv(algorithm+'_P_NS_given_X1pX2-BF-fit_an.csv',index=False,float_format='%.'+str(precision)+'f')

P_REM_given_X2_num_Bayes = P_REM_given_X2_num.iloc[:,1:]
P_REM_given_X2_num_Bayes.columns = range(len(Bayes_factors))
P_REM_given_X2_num_Bayes = P_REM_given_X2_num_Bayes.mul(Bayes_factors)
P_REM_given_X2_num_Bayes.rename(columns='P_REM('+Bayes['EOS']+')',inplace=True)
P_REM_given_X2_num_Bayes.insert(0,'X2',P_REM_given_X2_num['X2'])

P_REM_given_X2_num_Bayes.to_csv(algorithm+'_P_REM_given_X2-BF-num.csv',index=False,float_format='%.'+str(precision)+'f')

P_REM_given_X2_table_Bayes = P_REM_given_X2_table.iloc[:,1:]
P_REM_given_X2_table_Bayes.columns = range(len(Bayes_factors))
P_REM_given_X2_table_Bayes = P_REM_given_X2_table_Bayes.mul(Bayes_factors)
P_REM_given_X2_table_Bayes.rename(columns='P_REM('+Bayes['EOS']+')',inplace=True)
P_REM_given_X2_table_Bayes.insert(0,'X2',P_REM_given_X2_table['X2'])

P_REM_given_X2_table_Bayes.to_csv(algorithm+'_P_REM_given_X2-BF-fit.csv',index=False,float_format='%.'+str(precision)+'f')

P_REM_given_X2_table_an_Bayes = P_REM_given_X2_table_an.iloc[:,1:]
P_REM_given_X2_table_an_Bayes.columns = range(len(Bayes_factors))
P_REM_given_X2_table_an_Bayes = P_REM_given_X2_table_an_Bayes.mul(Bayes_factors)
P_REM_given_X2_table_an_Bayes.rename(columns='P_REM('+Bayes['EOS']+')',inplace=True)
P_REM_given_X2_table_an_Bayes.insert(0,'X2',P_REM_given_X2_table_an['X2'])

P_REM_given_X2_table_an_Bayes.to_csv(algorithm+'_P_REM_given_X2-BF-fit_an.csv',index=False,float_format='%.'+str(precision)+'f')